In [1]:
from google.colab import drive
drive.mount ('/content/gdrive' )
root_path = '/content/gdrive/My Drive/Retinopathy'
print(root_path)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/Retinopathy


In [2]:
!pip3 install tensorflow_addons

     |████████████████████████████████| 993kB 3.3MB/s 
     |████████████████████████████████| 421.8MB 43kB/s 
     |████████████████████████████████| 3.8MB 37.6MB/s 
     |████████████████████████████████| 450kB 61.3MB/s 
     |████████████████████████████████| 81kB 9.7MB/s 
ERROR: tensorboard 2.1.0 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.11.0 which is incompatible.
  Found existing installation: google-auth 1.4.2
    Uninstalling google-auth-1.4.2:
      Successfully uninstalled google-auth-1.4.2
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorflow 1.15.0
    Unins

In [0]:
import tensorflow as tf
import pathlib
import tensorflow.keras as k
import tensorflow_addons as tfa
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Dense,MaxPooling2D, Input, Conv2D, LeakyReLU, Flatten, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications.mobilenet import preprocess_input 
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import Model
import cv2
from tensorflow.python.framework.ops import disable_eager_execution

In [4]:
train_directory = root_path + "/OriginalImages/TrainingSet"
test_directory = root_path + "/OriginalImages/TestingSet"

training_set_path = pathlib.Path(train_directory)
testing_set_path = pathlib.Path(test_directory)
training_labels = root_path + "/GroundTruths/IDRiD_Disease Grading_Training Labels.csv"
test_labels = root_path + "/GroundTruths/IDRiD_Disease Grading_Testing Labels.csv"

print(training_set_path)
print(testing_set_path)
train_image_count = len(list(training_set_path.glob("*.jpg")))
test_image_count = len(list(testing_set_path.glob("*.jpg")))
print("train images =",train_image_count)
print("test images =",test_image_count)

/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet
/content/gdrive/My Drive/Retinopathy/OriginalImages/TestingSet
train images = 413
test images = 103


In [0]:
def decode_img(img):
    # convert the compressed string to a 3D uint8 tensor
    img = tf.image.decode_jpeg(img, channels=3)
    # Use `convert_image_dtype` to convert to floats in the [0,1] range.
    img = tf.image.convert_image_dtype(img, tf.float32)
    # resize the image to the desired size.
    return tf.image.resize(img, [256, 256])
def process_path(file_path,label):
    # load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label

def image_rotate(file_path,label):
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    img = tfa.image.rotate(img,90)
    return img, label

def adjust_contrast(file_path,label):
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    img = tf.image.adjust_contrast(img, 1.5)
    return img, label

def flipped_images(file_path, label):
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    img = tf.image.random_flip_left_right(img)
    return img, label

In [0]:
def referable_set(filepath,directory,startpoint,endpoint):
  images_list = []
  labels_list = []
  count = 0;
  with open(filepath) as labels:
    for line_no,line in enumerate(labels):
      if line_no > endpoint:
        break
      if (line_no >= startpoint) and (line_no <= endpoint):
        #print(line_no,line)
        x = line.split(',')
        #print(type(x),x)
        if float(x[1]) > 1 :
          labels_list.append(1)
          path = directory + "/"+x[0]+".jpg"
          images_list.append(path)
          count = count + 1
  #print(images_list,"\n")
  #print(labels_list)
  #print(count)
  return images_list,np.array(labels_list, dtype=np.float32)           


In [0]:
def non_referable_set(filepath,directory,startpoint,endpoint):
  images_list = []
  labels_list = []
  count = 0;
  with open(filepath) as labels:
    for line_no,line in enumerate(labels):
      if line_no > endpoint:
        break
      if (line_no >= startpoint) and (line_no <= endpoint):
        #print(line_no,line)
        x = line.split(',')
        #print(type(x),x)
        if float(x[1]) <= 1 :
          labels_list.append(0)
          path = directory + "/"+x[0]+".jpg"
          images_list.append(path)
          count = count + 1
  #print(images_list,"\n")
  #print(labels_list)
  #print(count)
  return images_list,np.array(labels_list, dtype=np.float32)  

In [8]:
valid_images_NR,valid_labels_NR = non_referable_set(training_labels,train_directory,1,193)
print(valid_images_NR, valid_labels_NR)

['/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_021.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_079.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_105.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_118.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_138.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_139.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_140.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_141.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_142.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_143.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_144.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_145.jpg', '/content/gdrive/My Drive/R

In [9]:
valid_images_R, valid_labels_R = referable_set(training_labels,train_directory, 1,51)
print(valid_images_R, valid_labels_R)

['/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_001.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_002.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_003.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_004.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_005.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_006.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_007.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_008.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_009.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_010.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_011.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_012.jpg', '/content/gdrive/My Drive/R

In [10]:

valid_images = valid_images_NR + valid_images_R
valid_labels = np.hstack((valid_labels_NR, valid_labels_R))
print(valid_labels)
print("Total valid images:",len(valid_images))


[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1.]
Total valid images: 100


In [11]:

validation_ds = tf.data.Dataset.from_tensor_slices((valid_images,valid_labels))

validation_ds = validation_ds.map(process_path, num_parallel_calls=None)
validation_ds = validation_ds.shuffle(100).batch(10)
for elem in validation_ds:
  print(elem)

(<tf.Tensor: shape=(10, 256, 256, 3), dtype=float32, numpy=
array([[[[0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.0004902 ],
         ...,
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.01568628]],

        [[0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.00637255],
         ...,
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.02892157],
         [0.        , 0.        , 0.00079657]],

        [[0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         ...,
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ]],

        ...,

        [[0.        , 0.        

In [12]:
train_images_NR, train_labels_NR = non_referable_set(training_labels,train_directory,194,413)
print(train_images_NR, train_labels_NR)

['/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_194.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_195.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_197.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_198.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_199.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_200.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_202.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_203.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_204.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_205.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_206.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_209.jpg', '/content/gdrive/My Drive/R

In [13]:
train_images_R, train_labels_R = referable_set(training_labels,train_directory, 52,413)
print(train_images_R, train_labels_R)

['/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_052.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_053.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_054.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_055.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_056.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_057.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_058.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_059.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_060.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_061.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_062.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_063.jpg', '/content/gdrive/My Drive/R

In [14]:
train_images = train_images_NR + train_images_R
train_labels = np.hstack((train_labels_NR, train_labels_R))
print(train_labels)
print("Total valid images:",len(train_images))

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1.]
Total valid images: 313


In [15]:

train_ds = tf.data.Dataset.from_tensor_slices((train_images,train_labels))
train_ds = train_ds.shuffle(313)
print(train_ds)
'''for elem in train_ds:
  print(elem)'''
train_ds = train_ds.map(process_path, num_parallel_calls=None)
for elem in train_ds:
  print(elem)

<ShuffleDataset shapes: ((), ()), types: (tf.string, tf.float32)>
(<tf.Tensor: shape=(256, 256, 3), dtype=float32, numpy=
array([[[0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        ...,
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ]],

       [[0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        ...,
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.01011029]],

       [[0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        ...,
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.01568628]],

     

In [0]:
train_img_contrast, train_label_contrast = non_referable_set(training_labels,train_directory,1,413)

In [0]:
train_contrast_ds = tf.data.Dataset.from_tensor_slices((train_img_contrast,train_label_contrast))
train_contrast_ds = train_contrast_ds.map(adjust_contrast, num_parallel_calls=None)
combined_training_ds= train_ds.concatenate(train_contrast_ds)
count = 0
#for elem in combined_training_ds:
  #count = count+1;
#print(count)

In [18]:
#flipped images take 104 from each set
flip_images = train_images_NR + train_images_R[:104]
flip_labels = np.hstack((train_labels_NR, train_labels_R[:104]))
print(len(flip_labels))
print("Total images to be flipped:",len(flip_images))
print(flip_images)

208
Total images to be flipped: 208
['/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_194.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_195.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_197.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_198.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_199.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_200.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_202.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_203.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_204.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_205.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_206.jpg', '/content/gdrive/My Drive/Retinopathy/OriginalImages/TrainingSet/IDRiD_2

In [19]:
train_flipped_ds = tf.data.Dataset.from_tensor_slices((flip_images,flip_labels))
train_flipped_ds = train_flipped_ds.shuffle(208)
train_flipped_ds = train_flipped_ds.map(flipped_images, num_parallel_calls=None)
combined_training_ds1= combined_training_ds.concatenate(train_flipped_ds)
count1 = 0
'''for elem in combined_training_ds1:
  count1 = count1+1
print(count1)'''

'for elem in combined_training_ds1:\n  count1 = count1+1\nprint(count1)'

In [0]:
train_ds = combined_training_ds1.shuffle(675).batch(10)


In [0]:

#test dataset creation
test_images_NR, test_labels_NR = non_referable_set(test_labels,test_directory,1,103)
test_images_R, test_labels_R = referable_set(test_labels,test_directory,1,103)


In [22]:

test_images = test_images_NR + test_images_R
test_labels = np.hstack((test_labels_NR, test_labels_R))
print(test_labels)
print("Total test images:",len(test_images))

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1.]
Total test images: 103


In [0]:
test_ds = tf.data.Dataset.from_tensor_slices((test_images,test_labels))
test_ds = test_ds.map(process_path, num_parallel_calls=None)
test_ds = test_ds.shuffle(103).batch(10)

In [24]:
batch_size = 100
epochs = 150
learning_rate = 0.0001

inputs = k.layers.Input(shape=(256,256,3))

conv_layer1 = Conv2D(32,3,(1,1))(inputs)
print(conv_layer1)
MP_layer1 = MaxPooling2D((3,3))(conv_layer1)
LLR_layer1 = LeakyReLU()(MP_layer1)

# a layer instance is callable on a tensor, and returns a tensor
conv_layer2 = Conv2D(64,3,(1,1),'same')(LLR_layer1)
MP_layer2 = MaxPooling2D((3,3))(conv_layer2)
LLR_layer2 = LeakyReLU()(MP_layer2)

conv_layer3 = Conv2D(128,3)(LLR_layer2)
MP_layer3 = MaxPooling2D((3,3))(conv_layer3)
LLR_layer3 = k.layers.LeakyReLU()(MP_layer3)


GAP = k.layers.GlobalAveragePooling2D()(LLR_layer3)

flatten = k.layers.Flatten()(GAP)
dense_layer1 = Dense(64)(flatten)
dense_layer2 = Dense(32)(dense_layer1)

output = k.layers.Dense(1, activation='sigmoid')(dense_layer1)
model = k.models.Model(inputs=inputs, outputs=output)
opt = k.optimizers.RMSprop(learning_rate)
model.compile(optimizer=opt ,loss="binary_crossentropy", metrics= ['accuracy'])
model.summary()
#model = Model(inputs=inputs, outputs=output)

Tensor("conv2d/Identity:0", shape=(None, 254, 254, 32), dtype=float32)
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 254, 254, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 84, 84, 32)        0         
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 84, 84, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 84, 84, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 28, 28, 64)        0         
________________________________________________________

In [25]:
filepath="/weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint_path = root_path + filepath

checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max',save_weights_only=False)
callbacks_list = [checkpoint]

model.fit(train_ds,validation_data=validation_ds, epochs = 150, callbacks=[checkpoint])

Train for 68 steps, validate for 10 steps
Epoch 1/150
64/68 [===========================>..] - ETA: 4s - loss: 0.6686 - accuracy: 0.6328
Epoch 00001: val_accuracy improved from -inf to 0.55000, saving model to /content/gdrive/My Drive/Retinopathy/weights-improvement-01-0.55.hdf5
68/68 [==============================] - 79s 1s/step - loss: 0.6641 - accuracy: 0.6311 - val_loss: 0.7049 - val_accuracy: 0.5500
Epoch 2/150
66/68 [============================>.] - ETA: 2s - loss: 0.5912 - accuracy: 0.7288
Epoch 00002: val_accuracy improved from 0.55000 to 0.58000, saving model to /content/gdrive/My Drive/Retinopathy/weights-improvement-02-0.58.hdf5
68/68 [==============================] - 76s 1s/step - loss: 0.5882 - accuracy: 0.7289 - val_loss: 0.6907 - val_accuracy: 0.5800
Epoch 3/150
65/68 [===========================>..] - ETA: 3s - loss: 0.5352 - accuracy: 0.7569
Epoch 00003: val_accuracy did not improve from 0.58000
68/68 [==============================] - 78s 1s/step - loss: 0.5352 - a

In [0]:
# summarize history for loss
plt.plot(model.history['loss'])
plt.plot(model.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()